In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
from tensorflow.keras.layers import ReLU,MaxPooling1D,Softmax,\
Bidirectional,LSTM,Conv2D,LeakyReLU,ELU,PReLU,Dense,Dropout
from tensorflow.keras.models import Sequential

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
df=pd.read_csv('Churn_Modelling.csv')

In [5]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [6]:
df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [7]:
X=df.iloc[:,3:13]
y=df.iloc[:,13]

In [8]:
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


In [9]:
X=pd.get_dummies(X,drop_first=True)

In [10]:
X

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,0,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1


In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=23)

In [12]:
from sklearn.preprocessing import StandardScaler

In [13]:
sc=StandardScaler()

In [14]:
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [17]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from tensorflow.keras.layers import Activation,Embedding,Flatten,BatchNormalization
from tensorflow.keras.activations import relu,sigmoid
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier,

In [33]:
def create_model(layers,activation):
    model=Sequential()
    for i, nodes in enumerate(layers):
        if i ==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
    model.add(Dense(units=1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [34]:
model=KerasClassifier(create_model)

C:\Users\viral\AppData\Local\Temp\ipykernel_16600\28813852.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model=KerasClassifier(create_model)


In [35]:
layers=[[20],[40,20],[45,30,15]]

In [36]:
activations=['sigmoid','relu']

In [37]:
param_grid=dict(layers=layers,activation=activations,batch_size=[128,256],epochs=[30])
param_grid

{'layers': [[20], [40, 20], [45, 30, 15]],
 'activation': ['sigmoid', 'relu'],
 'batch_size': [128, 256],
 'epochs': [30]}

In [38]:
grid=GridSearchCV(estimator=model,param_grid=param_grid,cv=5)

In [39]:
grid_result=grid.fit(X_train,y_train)

Epoch 1/30
47/47 [==============================] - 3s 7ms/step - loss: 0.9860 - accuracy: 0.2645
Epoch 2/30
47/47 [==============================] - 0s 7ms/step - loss: 0.7650 - accuracy: 0.4480
Epoch 3/30
47/47 [==============================] - 0s 7ms/step - loss: 0.6362 - accuracy: 0.6430
Epoch 4/30
47/47 [==============================] - 0s 7ms/step - loss: 0.5550 - accuracy: 0.7402
Epoch 5/30
47/47 [==============================] - 0s 7ms/step - loss: 0.5195 - accuracy: 0.7727
Epoch 6/30
47/47 [==============================] - 0s 7ms/step - loss: 0.4932 - accuracy: 0.7942
Epoch 7/30
47/47 [==============================] - 0s 7ms/step - loss: 0.4801 - accuracy: 0.7943
Epoch 8/30
47/47 [==============================] - 0s 7ms/step - loss: 0.4716 - accuracy: 0.7973
Epoch 9/30
47/47 [==============================] - 0s 7ms/step - loss: 0.4687 - accuracy: 0.7963
Epoch 10/30
47/47 [==============================] - 0s 7ms/step - loss: 0.4598 - accuracy: 0.8047
Epoch 11/30
47/47 [

47/47 [==============================] - 0s 7ms/step - loss: 0.4434 - accuracy: 0.8010
Epoch 23/30
47/47 [==============================] - 0s 6ms/step - loss: 0.4410 - accuracy: 0.8058
Epoch 24/30
47/47 [==============================] - 0s 6ms/step - loss: 0.4406 - accuracy: 0.8052
Epoch 25/30
47/47 [==============================] - 0s 6ms/step - loss: 0.4407 - accuracy: 0.8050
Epoch 26/30
47/47 [==============================] - 0s 6ms/step - loss: 0.4424 - accuracy: 0.8060
Epoch 27/30
47/47 [==============================] - 0s 6ms/step - loss: 0.4365 - accuracy: 0.8063
Epoch 28/30
47/47 [==============================] - 0s 6ms/step - loss: 0.4424 - accuracy: 0.8052
Epoch 29/30
47/47 [==============================] - 0s 6ms/step - loss: 0.4353 - accuracy: 0.8085
Epoch 30/30
12/12 [==============================] - 1s 6ms/step - loss: 0.4343 - accuracy: 0.8100
Epoch 1/30
47/47 [==============================] - 2s 6ms/step - loss: 0.5340 - accuracy: 0.7768
Epoch 2/30
47/47 [=====

47/47 [==============================] - 0s 8ms/step - loss: 0.4445 - accuracy: 0.8082
Epoch 13/30
47/47 [==============================] - 0s 8ms/step - loss: 0.4432 - accuracy: 0.8052
Epoch 14/30
47/47 [==============================] - 0s 8ms/step - loss: 0.4434 - accuracy: 0.8062
Epoch 15/30
47/47 [==============================] - 0s 8ms/step - loss: 0.4418 - accuracy: 0.8077
Epoch 16/30
47/47 [==============================] - 0s 7ms/step - loss: 0.4371 - accuracy: 0.8065
Epoch 17/30
47/47 [==============================] - 0s 7ms/step - loss: 0.4352 - accuracy: 0.8073
Epoch 18/30
47/47 [==============================] - 0s 8ms/step - loss: 0.4373 - accuracy: 0.8037
Epoch 19/30
47/47 [==============================] - 0s 8ms/step - loss: 0.4395 - accuracy: 0.8082
Epoch 20/30
47/47 [==============================] - 0s 8ms/step - loss: 0.4328 - accuracy: 0.8090
Epoch 21/30
47/47 [==============================] - 0s 8ms/step - loss: 0.4355 - accuracy: 0.8108
Epoch 22/30
47/47 [===

47/47 [==============================] - 0s 9ms/step - loss: 0.5143 - accuracy: 0.7918
Epoch 3/30
47/47 [==============================] - 0s 8ms/step - loss: 0.5014 - accuracy: 0.7942
Epoch 4/30
47/47 [==============================] - 0s 8ms/step - loss: 0.4899 - accuracy: 0.7932
Epoch 5/30
47/47 [==============================] - 0s 8ms/step - loss: 0.4837 - accuracy: 0.7948
Epoch 6/30
47/47 [==============================] - 0s 8ms/step - loss: 0.4740 - accuracy: 0.7942
Epoch 7/30
47/47 [==============================] - 0s 8ms/step - loss: 0.4694 - accuracy: 0.7957
Epoch 8/30
47/47 [==============================] - 0s 8ms/step - loss: 0.4643 - accuracy: 0.7977
Epoch 9/30
47/47 [==============================] - 0s 8ms/step - loss: 0.4605 - accuracy: 0.7970
Epoch 10/30
47/47 [==============================] - 0s 8ms/step - loss: 0.4517 - accuracy: 0.8017
Epoch 11/30
47/47 [==============================] - 0s 10ms/step - loss: 0.4568 - accuracy: 0.7940
Epoch 12/30
47/47 [=========

47/47 [==============================] - 0s 9ms/step - loss: 0.4406 - accuracy: 0.8022
Epoch 24/30
47/47 [==============================] - 0s 9ms/step - loss: 0.4390 - accuracy: 0.8008
Epoch 25/30
47/47 [==============================] - 0s 10ms/step - loss: 0.4358 - accuracy: 0.7998
Epoch 26/30
47/47 [==============================] - 0s 11ms/step - loss: 0.4371 - accuracy: 0.8017
Epoch 27/30
47/47 [==============================] - 0s 10ms/step - loss: 0.4360 - accuracy: 0.8047
Epoch 28/30
47/47 [==============================] - 0s 10ms/step - loss: 0.4356 - accuracy: 0.8010
Epoch 29/30
47/47 [==============================] - 0s 9ms/step - loss: 0.4350 - accuracy: 0.8000
Epoch 30/30
12/12 [==============================] - 1s 8ms/step - loss: 0.4427 - accuracy: 0.8013
Epoch 1/30
47/47 [==============================] - 3s 9ms/step - loss: 0.5779 - accuracy: 0.7300
Epoch 2/30
47/47 [==============================] - 0s 9ms/step - loss: 0.5328 - accuracy: 0.7903
Epoch 3/30
47/47 [==

...

24/24 [==============================] - 0s 8ms/step - loss: 0.5931 - accuracy: 0.7183
Epoch 5/30
24/24 [==============================] - 0s 8ms/step - loss: 0.5626 - accuracy: 0.7515
Epoch 6/30
24/24 [==============================] - 0s 7ms/step - loss: 0.5348 - accuracy: 0.7705
Epoch 7/30
24/24 [==============================] - 0s 7ms/step - loss: 0.5184 - accuracy: 0.7805
Epoch 8/30
24/24 [==============================] - 0s 7ms/step - loss: 0.5109 - accuracy: 0.7795
Epoch 9/30
24/24 [==============================] - 0s 7ms/step - loss: 0.4984 - accuracy: 0.7868
Epoch 10/30
24/24 [==============================] - 0s 8ms/step - loss: 0.4843 - accuracy: 0.7893
Epoch 11/30
24/24 [==============================] - 0s 7ms/step - loss: 0.4809 - accuracy: 0.7913
Epoch 12/30
24/24 [==============================] - 0s 11ms/step - loss: 0.4762 - accuracy: 0.7940
Epoch 13/30
24/24 [==============================] - 0s 11ms/step - loss: 0.4698 - accuracy: 0.7985
Epoch 14/30
24/24 [======

Epoch 25/30
24/24 [==============================] - 0s 11ms/step - loss: 0.3949 - accuracy: 0.8395
Epoch 26/30
24/24 [==============================] - 0s 11ms/step - loss: 0.3953 - accuracy: 0.8415
Epoch 27/30
24/24 [==============================] - 0s 11ms/step - loss: 0.3964 - accuracy: 0.8372
Epoch 28/30
24/24 [==============================] - 0s 11ms/step - loss: 0.3879 - accuracy: 0.8383
Epoch 29/30
24/24 [==============================] - 0s 11ms/step - loss: 0.3861 - accuracy: 0.8468
Epoch 30/30
6/6 [==============================] - 1s 8ms/step - loss: 0.3861 - accuracy: 0.8427
Epoch 1/30
24/24 [==============================] - 3s 12ms/step - loss: 0.7007 - accuracy: 0.5753
Epoch 2/30
24/24 [==============================] - 0s 11ms/step - loss: 0.5514 - accuracy: 0.7397
Epoch 3/30
24/24 [==============================] - 0s 12ms/step - loss: 0.5118 - accuracy: 0.7767
Epoch 4/30
24/24 [==============================] - 0s 11ms/step - loss: 0.4962 - accuracy: 0.7845
Epoch 5

24/24 [==============================] - 0s 13ms/step - loss: 0.4323 - accuracy: 0.8157
Epoch 15/30
24/24 [==============================] - 0s 12ms/step - loss: 0.4324 - accuracy: 0.8168
Epoch 16/30
24/24 [==============================] - 0s 12ms/step - loss: 0.4277 - accuracy: 0.8192
Epoch 17/30
24/24 [==============================] - 0s 14ms/step - loss: 0.4267 - accuracy: 0.8175
Epoch 18/30
24/24 [==============================] - 0s 12ms/step - loss: 0.4163 - accuracy: 0.8262
Epoch 19/30
24/24 [==============================] - 0s 12ms/step - loss: 0.4176 - accuracy: 0.8233
Epoch 20/30
24/24 [==============================] - 0s 13ms/step - loss: 0.4154 - accuracy: 0.8247
Epoch 21/30
24/24 [==============================] - 0s 11ms/step - loss: 0.4128 - accuracy: 0.8330
Epoch 22/30
24/24 [==============================] - 0s 11ms/step - loss: 0.4067 - accuracy: 0.8320
Epoch 23/30
24/24 [==============================] - 0s 10ms/step - loss: 0.4059 - accuracy: 0.8285
Epoch 24/30


24/24 [==============================] - 0s 11ms/step - loss: 0.5037 - accuracy: 0.7918
Epoch 4/30
24/24 [==============================] - 0s 11ms/step - loss: 0.4858 - accuracy: 0.7935
Epoch 5/30
24/24 [==============================] - 0s 11ms/step - loss: 0.4747 - accuracy: 0.7940
Epoch 6/30
24/24 [==============================] - 0s 11ms/step - loss: 0.4671 - accuracy: 0.7937
Epoch 7/30
24/24 [==============================] - 0s 11ms/step - loss: 0.4574 - accuracy: 0.7942
Epoch 8/30
24/24 [==============================] - 0s 11ms/step - loss: 0.4533 - accuracy: 0.7953
Epoch 9/30
24/24 [==============================] - 0s 11ms/step - loss: 0.4516 - accuracy: 0.7938
Epoch 10/30
24/24 [==============================] - 0s 11ms/step - loss: 0.4500 - accuracy: 0.7968
Epoch 11/30
24/24 [==============================] - 0s 11ms/step - loss: 0.4458 - accuracy: 0.7995
Epoch 12/30
24/24 [==============================] - 0s 11ms/step - loss: 0.4418 - accuracy: 0.8032
Epoch 13/30
24/24 

24/24 [==============================] - 0s 11ms/step - loss: 0.4258 - accuracy: 0.8228
Epoch 24/30
24/24 [==============================] - 0s 11ms/step - loss: 0.4276 - accuracy: 0.8217
Epoch 25/30
24/24 [==============================] - 0s 11ms/step - loss: 0.4181 - accuracy: 0.8243
Epoch 26/30
24/24 [==============================] - 0s 11ms/step - loss: 0.4197 - accuracy: 0.8205
Epoch 27/30
24/24 [==============================] - 0s 11ms/step - loss: 0.4134 - accuracy: 0.8248
Epoch 28/30
24/24 [==============================] - 0s 11ms/step - loss: 0.4118 - accuracy: 0.8305
Epoch 29/30
24/24 [==============================] - 0s 11ms/step - loss: 0.4068 - accuracy: 0.8327
Epoch 30/30
6/6 [==============================] - 1s 7ms/step - loss: 0.3577 - accuracy: 0.8580
Epoch 1/30
24/24 [==============================] - 3s 11ms/step - loss: 0.7303 - accuracy: 0.4495
Epoch 2/30
24/24 [==============================] - 0s 11ms/step - loss: 0.5993 - accuracy: 0.7347
Epoch 3/30
24/24 

In [40]:
grid_result.best_score_

0.855733335018158

In [41]:
grid_result.best_params_

{'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [45, 30, 15]}

In [52]:
classifire=Sequential()

In [53]:
classifire.add(Dense(units=20,kernel_initializer='he_normal',activation='relu',input_dim=11))

In [54]:
classifire.add(Dense(units=20,kernel_initializer='he_normal',activation='relu'))
classifire.add(Dense(units=15,kernel_initializer='he_normal',activation='relu'))

In [55]:
classifire.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

In [56]:
classifire.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [57]:
history=classifire.fit(X_train,y_train,validation_split=0.33,batch_size=10,epochs=100)

Epoch 1/100
503/503 [==============================] - 8s 11ms/step - loss: 0.4774 - accuracy: 0.7964 - val_loss: 0.4385 - val_accuracy: 0.8082
Epoch 2/100
503/503 [==============================] - 4s 9ms/step - loss: 0.4363 - accuracy: 0.8057 - val_loss: 0.4120 - val_accuracy: 0.8203
Epoch 3/100
503/503 [==============================] - 5s 9ms/step - loss: 0.4127 - accuracy: 0.8161 - val_loss: 0.3931 - val_accuracy: 0.8344
Epoch 4/100
503/503 [==============================] - 4s 8ms/step - loss: 0.3943 - accuracy: 0.8310 - val_loss: 0.3785 - val_accuracy: 0.8421
Epoch 5/100
503/503 [==============================] - 4s 8ms/step - loss: 0.3795 - accuracy: 0.8447 - val_loss: 0.3719 - val_accuracy: 0.8477
Epoch 6/100
503/503 [==============================] - 4s 8ms/step - loss: 0.3697 - accuracy: 0.8479 - val_loss: 0.3698 - val_accuracy: 0.8485
Epoch 7/100
503/503 [==============================] - 4s 8ms/step - loss: 0.3608 - accuracy: 0.8545 - val_loss: 0.3613 - val_accuracy: 0.856

Epoch 58/100
503/503 [==============================] - 4s 8ms/step - loss: 0.2712 - accuracy: 0.8846 - val_loss: 0.4050 - val_accuracy: 0.8267
Epoch 59/100
503/503 [==============================] - 6s 12ms/step - loss: 0.2695 - accuracy: 0.8857 - val_loss: 0.4148 - val_accuracy: 0.8437
Epoch 60/100
503/503 [==============================] - 4s 9ms/step - loss: 0.2692 - accuracy: 0.8824 - val_loss: 0.4035 - val_accuracy: 0.8401
Epoch 61/100
503/503 [==============================] - 3s 7ms/step - loss: 0.2682 - accuracy: 0.8838 - val_loss: 0.3987 - val_accuracy: 0.8340
Epoch 62/100
503/503 [==============================] - 4s 7ms/step - loss: 0.2685 - accuracy: 0.8854 - val_loss: 0.4108 - val_accuracy: 0.8465
Epoch 63/100
503/503 [==============================] - 3s 7ms/step - loss: 0.2654 - accuracy: 0.8859 - val_loss: 0.4106 - val_accuracy: 0.8393
Epoch 64/100
503/503 [==============================] - 3s 6ms/step - loss: 0.2634 - accuracy: 0.8859 - val_loss: 0.4153 - val_accuracy

In [58]:
# from sklearn.ensemble import RandomforestClassifire

In [59]:
y_pred=classifire.predict(X_test)

79/79 [==============================] - 1s 5ms/step


In [60]:
y_pred

array([[0.00183819],
       [0.5862606 ],
       [0.99976957],
       ...,
       [0.6883494 ],
       [0.04603062],
       [0.00316595]], dtype=float32)

In [61]:
for i in range(len(y_pred)):
    if y_pred[i] >0.5:
        y_pred[i]=1
    else:
        y_pred[i]=0

In [62]:
y_pred

array([[0.],
       [1.],
       [1.],
       ...,
       [1.],
       [0.],
       [0.]], dtype=float32)

In [65]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
cm=confusion_matrix(y_test,y_pred)

In [64]:
cm

array([[1842,  156],
       [ 250,  252]], dtype=int64)

In [66]:
score=accuracy_score(y_test,y_pred)

In [67]:
score

0.8376

In [68]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.92      0.90      1998
           1       0.62      0.50      0.55       502

    accuracy                           0.84      2500
   macro avg       0.75      0.71      0.73      2500
weighted avg       0.83      0.84      0.83      2500



In [69]:
y_test.value_counts()

0    1998
1     502
Name: Exited, dtype: int64